## Step 1
Load model response

In [1]:
import json

# Path to your JSONL file
file_path = "data4verification/iGSM_verify_1000_cot.jsonl"
response_list = []
gt_response_list = []

with open(file_path, 'r') as file:
    for line in file:
        obj = json.loads(line)  # Parse each line as a JSON object
        response_list.append(obj)
        # print(obj.keys())  # Process or print each object
        # print(obj['idx'])
        
gt_cot_filepath = "data4verification/iGSM_verify_1000.jsonl"
with open(gt_cot_filepath, 'r') as file:
    for line in file:
        obj = json.loads(line)  # Parse each line as a JSON object
        gt_response_list.append(obj)
        # print(obj.keys())  # Process or print each object
        # print(obj['idx'])



## Step 2
Calculate PRM based on responses and DAGs.

In [ ]:
import random
import numpy as np
from igsm_gym.generation import ProblemGenerator
from igsm_gym.utils.misc import seed_all
from igsm_gym.env import RewardModel
import json
import copy

config = {
    "english_path": "igsm_gym/generation/english/categorization.json",
    "max_operations": 21,
    "force": False,
}

seed = 10
seed_all(seed)
print(seed)

test_num = 1000
data = []
Gd_list = []

pg = ProblemGenerator(config=config,debug=True)

for i in range(test_num):
    while not pg.draw_question():
        continue
    print("index:", i)
    
    question = pg.generate_question()
    answer = pg.generate_answer()
    gt_cot = answer
    gt_answer = pg.Gd.topo[-1].value
    
    response = response_list[i]['code'][0]
    
    reward_model = RewardModel(Gd=pg.Gd)
    PRM_reward = reward_model.PRM(
        llm_output=response
    )
    
    print(question)
    print("@"*10)
    print("gt cot:")
    print(gt_cot)
    print("gt answer:")
    print(gt_answer)
    print("@"*10)
    print("respones:")
    print(response)
    print("@"*10)
    print("PRM:")
    print(PRM_reward)
    print("="*30)

    
    
# Save the data to a JSONL file
# output_file = "iGSM_verify_{}.jsonl".format(test_num)
# with open(output_file, "w") as f:
#     for entry in data:
#         f.write(json.dumps(entry) + "\n")  # Convert each entry to a JSON string and write it as a new line